In [ ]:
import json
from tqdm import tqdm

In [ ]:
q = []
with open("HC3/all.jsonl", "r", encoding="utf-8") as f:
    for line in f.readlines():
        line = json.loads(line)
        q.append(line["question"].replace('\n', ' ').replace('\t', ' '))
print(len(q))
print(q[1])

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-1b7")
model = AutoModelForCausalLM.from_pretrained("bigscience/bloomz-1b7", torch_dtype="auto", device_map="auto")

with open("HC3/bloomz-1b7", "w", encoding="utf-8") as w:
    for i in tqdm(range(len(q))):
        text = q[i]
        inputs = tokenizer.encode(text, return_tensors="pt").to("cuda")
        outputs = model.generate(inputs)
        output = tokenizer.decode(outputs[0])
        w.write(output)
        w.write('\n')


In [26]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2', device=0)
set_seed(42)

with open("HC3/gpt2", "w", encoding="utf-8") as w:
    for i in tqdm(range(len(q))):
        try:
            text = q[i]
            output = generator(text, min_length=64, max_length=512)
        except Exception as e:
            output = f"{i} error: {e}"
        finally:
            w.write(str(output))
            w.write('\n')


In [ ]:
from transformers import pipeline
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B', device=0)

with open("HC3/gpt-neo-1.3B", "w", encoding="utf-8") as w:
    for i in tqdm(range(len(q))):
        text = q[i]
        output = generator(text, min_length=64, max_length=512)
        w.write(output)
        w.write('\n')